# Account Tampering - Suspicious Failed Logon Reasons
This method uses uncommon error codes on failed logons to determine suspicious activity and tampering with accounts that have been disabled or somehow restricted.

## Rule Content
```
- title: Account Tampering - Suspicious Failed Logon Reasons
  id: 9eb99343-d336-4020-a3cd-67f3819e68ee
  description: This method uses uncommon error codes on failed logons to determine
    suspicious activity and tampering with accounts that have been disabled or somehow
    restricted.
  author: Florian Roth
  modified: 2019/03/01
  references:
  - https://twitter.com/SBousseaden/status/1101431884540710913
  tags:
  - attack.persistence
  - attack.privilege_escalation
  - attack.t1078
  logsource:
    product: windows
    service: security
    category: null
  detection:
    selection:
      EventID:
      - 4625
      - 4776
      Status:
      - '0xC0000072'
      - '0xC000006F'
      - '0xC0000070'
      - '0xC0000413'
      - '0xC000018C'
      - '0xC000015B'
    condition: selection
  falsepositives:
  - User using a disabled account
  level: high

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-endpoint-winevent-security-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='(event_id:("4625" OR "4776") AND event_status:("0xC0000072" OR "0xC000006F" OR "0xC0000070" OR "0xC0000413" OR "0xC000018C" OR "0xC000015B"))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()